In [12]:
from anthropic import AnthropicBedrock
import random
from tqdm import tqdm
import os
import numpy as np
import re
client = AnthropicBedrock(
    aws_region="us-west-2",
)


In [4]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 250655.62it/s]


In [5]:
examples = {
    '你微多少': ['这句话使用了简称“微”来代替“微信”，并且询问“多少”，即询问对方的微信号码。这是非常常见的一种隐晦表达方式，用以规避系统对敏感词的审核。在实际交流中，这种说法通常被理解为询问对方微信并希望得到对方的微信联系方式。', 'Y'],
    '恩，微信多少加你': ['这句话虽然使用了“微信多少”这样的表述，但实际上是在询问对方的微信号码，并含有添加对方为微信好友的意思。"加你"进一步明确了这一意图。因此，这句话可以被视为一句邀请加微信的话语。', 'Y'],
    '可以互相的留个v吗？后面你上线做任务的时候你和我说的': ['这句话中，“留个v”通常指的是留下微信联系方式。"v"在这里是微信的谐音，因为微信的英文名是WeChat，其中的"We"与"v"谐音。此外，句中提到“后面你上线做任务的时候你和我说的”暗示了后续的沟通和联系，这通常是在加了微信之后才会进行的行为。因此，这句话可以判断为邀请加微信的话语。', 'Y'],
    '方便给个微信吗？一个朋友想回来玩，你们自己谈谈？我就懒得传话了': ['这句话虽然使用了“给个微信”这样的直接表达，但是从上下文来看，“一个朋友想回来玩，你们自己谈谈？”以及“我就懒得传话了”都暗示了通过微信进行联系和沟通的意图。因此，可以判断这是一句邀请加微信的话语。', 'Y'],
    '你微信叫啥': ['这句话虽然直接使用了“微信”这个词，但是通过询问对方的微信昵称或账号，实际上是在暗示或者寻求添加微信好友的邀请。在社交场合中，这种问法通常是建立进一步联系的第一步。', 'Y'],
    '微信发我我加一下': ['这句话虽然简单，但是直接表达了想要添加对方为微信好友的意图。"微信发我"通常意味着通过微信发送信息，而"我加一下"则明确表示希望加入到对方的微信联系人中。这里的表述已经足够清晰，可以判断为邀请加微信的话语。', 'Y'],
    'P19372000230加这个微信': ['这句话中直接出现了“微信”这个词，并且前面有数字“P19372000230”，这看起来像是一个微信号或者某种形式的微信联系方式的代码。因此，可以判断这是一句邀请加微信的话语。', 'Y'],
    '有意十微': ['这句话使用了谐音字“十”代替了“加”，结合后面的“微”，可以推断出这是“加微信”的意思。"有意"则可能表示对方有意愿进行进一步的联系或交流。', 'Y'],
    '莱-玩-扥-呺嘛？ 琎-僦-五-千真-充-- 天天6-4-8真-充还有价-值万-元豪礼 嗖下威：S D S J L U 详细说': ['这句话虽然使用了谐音字和符号来代替敏感词汇，但是通过“莱-玩-扥-呺嘛？”这部分可以推测出是在询问对方是否愿意添加某种联系方式。“琎-僦-五-千真-充--”和后面的数字“6-4-8”以及“价值万元豪礼”这些信息通常出现在一些推广或者诱导添加微信的语境中。最后的“嗖下威：S D S J L U 详细说”中的“威”很可能是微信的谐音，“S D S J L U”可能是一个微信号或者某种形式的联系方式。综合这些信息，这句话很可能是邀请加微信的话语。', 'Y'],
    '来玩扥号 吗？ 进就送伍仟， 后续每天648茺 有兴趣加微：hgx奇55（奇=7）': ['这句话使用了谐音和替代的方式来规避审核。其中，“扥号”很可能是“账号”的谐音，而“微：hgx奇55（奇=7）”中的“微”指的是微信，后面跟着的疑似是一串微信号，并且用“奇=7”这样的方式来表示数字“7”，这是常见的规避手段。整个句子表达了邀请对方加入微信，进行游戏活动的意思。', 'Y'],
    '"你为多少啊，我让组长加你一下。\n你回多少啊，我让组长加你一下。': ['这句话使用了谐音的方式来规避审核。使用了“为”代替微信的“微”。', 'Y'],
    '那我邀你进剑侠郡，\\/留个我搜你 方便一起组队': ['这句话通过“\\/留个我搜你”这种方式来暗示交换微信信息。"搜你"可能指的是在微信中搜索对方微信号，以便添加为好友。"方便一起组队"则是为了加强这种社交目的，在游戏中组队通常需要通过微信等通讯工具进行联系。整体来看，这句话是在隐晦地提出加微信的邀请。', 'Y'],
    '徽多少我加你，发给你换': ['这句话使用了“徽”代替了“微”，并且“加你”表达了添加对方的意思，“发给你换”可能意味着发送微信好友请求。整体来看，这句话是在隐晦地表达想要添加对方为微信好友的意愿。', 'Y'],
    '你徵多少': ['虽然这句话“你徵多少”并没有直接表达出加微信的意思。但“徵”字在这里很有可能是被用来代替“微信”的“微”。', 'Y'],
    '进一下咱们家族的那个讨论组啊': ['这句话使用了“家族的那个讨论组”来代替“微信群”，是一种隐晦的表达方式。在日常生活中，讨论组往往可以通过微信等社交平台进行，而“家族”的提法也可能意味着这是一个私人群组。因此，这句话实际上很可能是邀请对方加入一个微信群的委婉说法。', 'Y'],
    '我转锤子天王，还有补贴嘛，这么多木我玩锤子了': ['这句话并没有直接或者隐晦地表达出加微信的意思。这句话中的“转锤子天王”和“玩锤子了”可能是方言或者网络用语，但它们并不构成通常用来暗示加微信的语境或表述。此外，“补贴”一词在这里也没有明显的指向加微信的含义。因此，这句话不符合邀请加微信的话语特征。', 'N'],
    '你可以玩一下战忍或者棍少': ['这句话并没有直接或者隐晦地表达出加微信的意思。这句话看起来像是在建议对方玩某个游戏或者应用，并没有涉及到社交联系方式。如果这句话中包含有微信的谐音字、形近字或相关符号，那么可能意味着邀请加微信，但就这句话本身而言，并不符合邀请加微信的特征。', 'N'],
    '下周二晚上估计你能升级，到时候还要麻烦你': ['这句话并没有直接或隐晦地表达出加微信的意思。它看起来像是在讨论某个计划或者活动，并提到了“升级”和“麻烦你”，但没有使用任何与微信相关的谐音、形近字或符号，也没有其他表述暗示加微信。', 'N'],
    '<嘉峪关(197,258)>云儿在这': ['这句话并没有直接或隐晦地表达出加微信的意思。它看起来像是一个地名和一个人名的组合，并没有使用任何谐音字、形近字或符号来代替“微信”这个词，也没有其他表述暗示加微信。因此，这句话不符合邀请加微信的话语特征。', 'N'],
    '在下【凉面自然辣】，初来乍到，还请诸位多多包涵。#65#65': ['这句话并没有直接或者隐晦地表达出加微信的意思。它看起来像是一个人在介绍自己，并表达了希望得到别人理解的愿望。"凉面自然辣"可能是这个人的昵称或者网名，而数字串#65#65在当前上下文中并不具有明显的加微信的含义。因此，这句话不符合邀请加微信的话语特征。', 'N'],
    '我升级以后可以换8g的。97级的腰带，我就可以想到最高级了。': ['这句话并没有直接或隐晦地表达出加微信的意思。它看起来像是在讨论某个游戏或者等级系统中关于升级和装备的话题，并没有明显的社交联系方式交换的暗示。此外，也没有使用谐音字、形近字或符号来代替“微信”这个词。因此，这句话不是一句邀请加微信的话语。', 'N'],
    '友情提醒:家族群发公告让你加群的注意了群里多个骗子配合 先混熟,在发你木马链接 咱们玩游戏主打一个不加任何群 就不会被骗  别进广告家族': ['这句话并不是一个邀请加微信的话语。这句话实际上是在提醒人们注意，不要轻易加入不熟悉的群组，因为里面可能有骗子。它强调了不要点击不明链接，也不要加入所谓的“广告家族”，这是一种安全提示，而不是邀请添加微信好友。', 'N'],
    '刚出了个火，你赶紧去': ['这句话“刚出了个火，你赶紧去”并没有明显的表示加微信的意思。这里的“火”可能指的是某个事件或者情况，而“你赶紧去”则像是催促对方去做某事，但并没有提到加微信或者类似的暗示。如果这句话中包含有隐晦的加微信的意思，那么它使用的表达方式并不直接，不符合通常的邀请加微信的语境。', 'N'],
    '在下【三号大哥】，初来乍到，还请诸位多多包涵。#65#65': ['这句话没有直接或者隐晦地表达出加微信的意思。它看起来像是一个人在论坛或者群组中的自我介绍，并表达了希望得到其他人包容的意愿。"#65#65"可能是用来分隔内容的符号，并不具有加微信的含义。因此，这句话不是一句邀请加微信的话语。', 'N'],
    '偷到一个野怪，但是你不要招到有地有那个模式的地方，不然回头搞不好一堆人来抽死你。': ['这句话并没有直接或隐晦地表达出加微信的意思。这句话看起来像是在描述一个游戏场景，提醒对方不要去某个地方，否则可能会有危险。没有使用任何谐音字、形近字或符号来代替“微信”，也没有其他表述暗示加微信。', 'N'],
    '出师吧，给你发任务经验会更多': ['这句话并没有直接或隐晦地表达出加微信的意思。这里的“出师”和“发任务经验”看起来更像是游戏或者某种任务中的语境，而不是要求添加微信的社交邀请。如果这句话中包含有谐音字或形近字，它们并没有明显指向“微信”这个词。因此，这句话不符合邀请加微信的话语特征。', 'N'],
    '什么啊都没头没脑的': ['这句话“什么啊都没头没脑的”并没有直接或者隐晦地表达出加微信的意思。它看起来更像是一句表达困惑或者不理解的话，与加微信无关。如果这句话中包含有谐音字、形近字或符号来代替“微信”这个词，那么在当前的形式下，并没有体现出这样的替代。因此，这句话不符合邀请加微信的话语特征。', 'N'],
    '我们以前是排队的，底价购买': ['这句话并没有直接或隐晦地表达出加微信的意思。它看起来像是在谈论过去的某个购买活动或者行为，并没有明显的邀请交换联系方式或加微信的意图。', 'N'],
    '岛求组': ['这句话并没有直接或隐晦地表达出加微信的意思，并没有明显的邀请交换联系方式或加微信的意图。', 'N'],
    '月岛组一个': ['这句话并没有直接或隐晦地表达出加微信的意思，并没有明显的邀请交换联系方式或加微信的意图。', 'N'],
    '#2多少收': ['这句话并没有直接或隐晦地表达出加微信的意思，并没有明显的邀请交换联系方式或加微信的意图。', 'N'],
}
examples_prompt_list = []
for k,v in examples.items():
    examples_prompt_list.append(f'<example>\n<sentence>\n{k}\n</sentence>\n<explanation>\n{v[0]}\n</explanation>\n<result>\n{v[1]}\n</result>\n</example>')

In [6]:
embeddings = model.encode(list(examples.keys()), 
                            batch_size=12, 
                            max_length=512,
                            )['dense_vecs']

In [7]:
prompt = '''
你是一位文字内容审核员,需要仔细分析给定句子是否含有邀请加微信的暗示。请按以下步骤分析:

1. 查看句子是否包含"微信"或“微”这个词或其谐音字、形近字等变体。
2. 检查句子中是否使用了符号或其他隐晦方式暗示"微信"。
3. 结合上下文语义,判断句子的真正意图是否为邀请加微信。

以下是一些示例,帮助你更好地理解邀请加微信的常见表达方式:
<examples>
{examples_prompt}
</examples>

现在请分析以下句子:
<sentence>
{text}
</sentence>

在<explanation></explanation>标签中用一句话解释你的分析过程，在<result></result>标签中输出Y或N,表示该句子是否在邀请加微信。
'''.strip()

In [8]:
import pandas as pd
df = pd.read_excel('ai返回结果配对0321.xlsx')
text_list = df['文本']

In [13]:
for text_id, text in enumerate(tqdm(text_list)):
    embedding = model.encode(text)['dense_vecs']
    similarity = embedding @ embeddings.T
    examples_prompt = '\n\n'.join([examples_prompt_list[example_idx] for example_idx in np.argsort(similarity)[-5:]])
    message = client.messages.create(
        model="anthropic.claude-3-haiku-20240307-v1:0",
        max_tokens=512,
        messages=[{"role": "user", "content": prompt.format(examples_prompt = examples_prompt, text=text)}]
    )
    pattern = r'<result>(.*?)</result>'
    result = re.findall(pattern, message.content[0].text, re.DOTALL)[0]
    # with open(f'{text_id:05d}.txt', 'w') as f:
    #     f.write(result)
    
    break

  0%|          | 0/10610 [00:01<?, ?it/s]
